In [1]:
import pandas as pd
import numpy as np
import os
from keras.models import Sequential,Model
from keras.layers import *

from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau
from keras.models import load_model
import keras
from keras import backend as K

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
import time
from imageloader import data_process
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sn
import copy
from keras.preprocessing import sequence

Using TensorFlow backend.


In [2]:
#reduce_lr_loss = ReduceLROnPlateau(monitor='loss', factor=0.1, patience=patience_lr, verbose=1, epsilon=1e-4, mode='min')
lr_reduce = keras.callbacks.ReduceLROnPlateau(monitor='val_acc',factor=0.5, patience=20)
#model save
MODEL_SAVE_FOLDER_PATH = '../model/lstm'
if not os.path.exists(MODEL_SAVE_FOLDER_PATH):
  os.mkdir(MODEL_SAVE_FOLDER_PATH)

model_path = MODEL_SAVE_FOLDER_PATH + '.hdf5'

cb_checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_acc',
                                verbose=1, save_best_only=True)


#early_stopping = EarlyStopping()

# Minmax 
scaler = MinMaxScaler()
scaler_ = MinMaxScaler()

# cudnn 오류 해결용(RTX에서만 생기는 문제로 보임))
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)


In [3]:
number = False

if(number):
    dp = data_process('./DATA/aug/all/train/number',number)
    dp.point_data_load()
    #dp.image_read()
    dp.sequence_50()
    dp.image_make()
    dp.data_shuffle()
  
    class_num = 10
  
else:
    dp_train = data_process('./DATA/aug/all/train/Alphabet',number)
    dp_train.point_data_load()
    #dp.image_read()
    dp_train.sequence_50()
    dp_train.image_make()
    dp_train.data_shuffle()
    train_len = np.size(dp_train.point,0)
    print(train_len)
  
    
    dp_test = data_process('./DATA/aug/all/test/Alphabet',number)
    dp_test.point_data_load()
    #dp.image_read()
    dp_test.sequence_50()
    dp_test.image_make()
    dp_test.data_shuffle()
    test_len = np.size(dp_test.point,0)
  
    class_num = 26

--- 14.845260381698608 seconds ---
0
7047
--- 2.873248338699341 seconds ---
0


In [4]:
# minmax 실행
train_x = dp_train.point[:,:,0].reshape((-1,1))
test_x = dp_test.point[:,:,0].reshape((-1,1))

train_y = dp_train.point[:,:,1].reshape((-1,1))
test_y = dp_test.point[:,:,1].reshape((-1,1))

data_x = scaler.fit_transform(train_x).reshape((-1,64,1))
data_test_x = scaler.transform(test_x).reshape((-1,64,1))

data_y = scaler_.fit_transform(train_y).reshape((-1,64,1))
data_test_y = scaler_.transform(test_y).reshape((-1,64,1))

data = np.hstack((data_x,data_y)).reshape((-1,64,2),order='F')
data_test = np.hstack((data_test_x,data_test_y)).reshape((-1,64,2),order='F')

#train set test set 나누기
X_train = data[:,:,:]
#X_train_ = copy.deepcopy(dp.point)
X_test = data_test[:,:,:]
X_test_ = copy.deepcopy(data[:,:,:])

X_train_img = dp_train.images[:]
X_test_img = dp_test.images[:]
Y_train = dp_train.label[:]
Y_test = dp_test.label[:]


Y_train = keras.utils.to_categorical(Y_train,num_classes=class_num, dtype='float32')
Y_test = keras.utils.to_categorical(Y_test,num_classes=class_num, dtype='float32')


/home/kist-student/anaconda3/envs/tensor/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/home/kist-student/anaconda3/envs/tensor/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [5]:
# cudnn 오류 해결용(RTX에서만 생기는 문제로 보임))
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

In [6]:
input_2 = Input(shape=(64, 2))

x_2 = Conv1D(32,kernel_size=4, padding='same',strides=1)(input_2)
x_2 = BatchNormalization(momentum=0.8)(x_2)
x_2 = LeakyReLU(0.2)(x_2)
x_2 = Dropout(0.5)(x_2)

x_2 = Conv1D(64,kernel_size=5, strides=1, padding='same')(x_2)
x_2 = BatchNormalization(momentum=0.8)(x_2)
x_2 = LeakyReLU(alpha=0.2)(x_2)
x_2 = Dropout(0.5)(x_2)

x_2 = Conv1D(128, kernel_size=6, strides=1, padding='same')(x_2)
x_2 = BatchNormalization(momentum=0.8)(x_2)
x_2 = LeakyReLU(0.2)(x_2)
x_2 = Dropout(0.5)(x_2)


x_2 = Conv1D(256, kernel_size=7, strides=1, padding='same')(x_2)
x_2 = BatchNormalization(momentum=0.8)(x_2)
x_2 = LeakyReLU(0.2)(x_2)
x_2 = Dropout(0.5)(x_2)

x_2 = Conv1D(512, kernel_size=8, strides=1, padding='same')(x_2)
x_2 = BatchNormalization(momentum=0.8)(x_2)
x_2 = LeakyReLU(0.2)(x_2)
x_2 = Dropout(0.5)(x_2)

x_2 = GlobalAveragePooling1D()(x_2)

x_2 = Dense(class_num,activation='softmax')(x_2)

model = Model(inputs=input_2, outputs = x_2)



Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [7]:
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64, 2)             0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 64, 32)            288       
_________________________________________________________________
batch_normalization_1 (Batch (None, 64, 32)            128       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 64, 32)            0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 64, 32)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 64, 64)            10304     
_________________________________________________________________
batch_normalization_2 (Batch (None, 64, 64)            256       
__________

In [8]:
model.compile(loss='categorical_crossentropy',
                optimizer=keras.optimizers.Adam(lr=0.001,epsilon=1e-04),
                metrics=['accuracy'])

results = model.fit(X_train,Y_train,epochs=250,batch_size=128,validation_data=(X_test,Y_test),callbacks=[cb_checkpoint,lr_reduce])


Instructions for updating:
Use tf.cast instead.
Train on 7047 samples, validate on 1393 samples
Epoch 1/250
7047/7047 [==============================] - 5s 686us/step - loss: 2.0246 - acc: 0.3988 - val_loss: 2.1381 - val_acc: 0.2850

Epoch 00001: val_acc improved from -inf to 0.28500, saving model to ../model/lstm.hdf5
Epoch 2/250
7047/7047 [==============================] - 1s 174us/step - loss: 1.2181 - acc: 0.6406 - val_loss: 1.3696 - val_acc: 0.5879

Epoch 00002: val_acc improved from 0.28500 to 0.58794, saving model to ../model/lstm.hdf5
Epoch 3/250
7047/7047 [==============================] - 1s 168us/step - loss: 0.8306 - acc: 0.7731 - val_loss: 0.9040 - val_acc: 0.7617

Epoch 00003: val_acc improved from 0.58794 to 0.76167, saving model to ../model/lstm.hdf5
Epoch 4/250
7047/7047 [==============================] - 1s 173us/step - loss: 0.5512 - acc: 0.8555 - val_loss: 0.7974 - val_acc: 0.7681

Epoch 00004: val_acc improved from 0.76167 to 0.76813, saving model to ../model/lstm.

7047/7047 [==============================] - 1s 170us/step - loss: 0.0720 - acc: 0.9774 - val_loss: 0.2306 - val_acc: 0.9397

Epoch 00040: val_acc did not improve from 0.94975
Epoch 41/250
7047/7047 [==============================] - 1s 166us/step - loss: 0.0506 - acc: 0.9847 - val_loss: 0.2308 - val_acc: 0.9383

Epoch 00041: val_acc did not improve from 0.94975
Epoch 42/250
7047/7047 [==============================] - 1s 176us/step - loss: 0.0469 - acc: 0.9851 - val_loss: 0.2319 - val_acc: 0.9361

Epoch 00042: val_acc did not improve from 0.94975
Epoch 43/250
7047/7047 [==============================] - 1s 173us/step - loss: 0.0405 - acc: 0.9889 - val_loss: 0.2165 - val_acc: 0.9318

Epoch 00043: val_acc did not improve from 0.94975
Epoch 44/250
7047/7047 [==============================] - 1s 171us/step - loss: 0.0438 - acc: 0.9868 - val_loss: 0.2534 - val_acc: 0.9261

Epoch 00044: val_acc did not improve from 0.94975
Epoch 45/250
7047/7047 [==============================] - 1s 179us/s

7047/7047 [==============================] - 1s 177us/step - loss: 0.0243 - acc: 0.9926 - val_loss: 0.2073 - val_acc: 0.9426

Epoch 00083: val_acc did not improve from 0.95477
Epoch 84/250
7047/7047 [==============================] - 1s 173us/step - loss: 0.0262 - acc: 0.9916 - val_loss: 0.2104 - val_acc: 0.9397

Epoch 00084: val_acc did not improve from 0.95477
Epoch 85/250
7047/7047 [==============================] - 1s 178us/step - loss: 0.0308 - acc: 0.9899 - val_loss: 0.2479 - val_acc: 0.9282

Epoch 00085: val_acc did not improve from 0.95477
Epoch 86/250
7047/7047 [==============================] - 1s 179us/step - loss: 0.0269 - acc: 0.9916 - val_loss: 0.2256 - val_acc: 0.9454

Epoch 00086: val_acc did not improve from 0.95477
Epoch 87/250
7047/7047 [==============================] - 1s 173us/step - loss: 0.0357 - acc: 0.9877 - val_loss: 0.2807 - val_acc: 0.9182

Epoch 00087: val_acc did not improve from 0.95477
Epoch 88/250
7047/7047 [==============================] - 1s 189us/s

7047/7047 [==============================] - 1s 171us/step - loss: 0.0251 - acc: 0.9919 - val_loss: 0.2658 - val_acc: 0.9253

Epoch 00126: val_acc did not improve from 0.95477
Epoch 127/250
7047/7047 [==============================] - 1s 182us/step - loss: 0.0204 - acc: 0.9943 - val_loss: 0.3121 - val_acc: 0.9167

Epoch 00127: val_acc did not improve from 0.95477
Epoch 128/250
7047/7047 [==============================] - 1s 187us/step - loss: 0.0217 - acc: 0.9935 - val_loss: 0.2509 - val_acc: 0.9268

Epoch 00128: val_acc did not improve from 0.95477
Epoch 129/250
7047/7047 [==============================] - 1s 174us/step - loss: 0.0194 - acc: 0.9947 - val_loss: 0.2437 - val_acc: 0.9368

Epoch 00129: val_acc did not improve from 0.95477
Epoch 130/250
7047/7047 [==============================] - 1s 172us/step - loss: 0.0202 - acc: 0.9940 - val_loss: 0.2330 - val_acc: 0.9282

Epoch 00130: val_acc did not improve from 0.95477
Epoch 131/250
7047/7047 [==============================] - 1s 17

7047/7047 [==============================] - 1s 181us/step - loss: 0.0193 - acc: 0.9939 - val_loss: 0.1983 - val_acc: 0.9454

Epoch 00169: val_acc did not improve from 0.95836
Epoch 170/250
7047/7047 [==============================] - 1s 173us/step - loss: 0.0190 - acc: 0.9942 - val_loss: 0.2196 - val_acc: 0.9404

Epoch 00170: val_acc did not improve from 0.95836
Epoch 171/250
7047/7047 [==============================] - 1s 171us/step - loss: 0.0228 - acc: 0.9930 - val_loss: 0.2535 - val_acc: 0.9318

Epoch 00171: val_acc did not improve from 0.95836
Epoch 172/250
7047/7047 [==============================] - 1s 175us/step - loss: 0.0213 - acc: 0.9939 - val_loss: 0.2001 - val_acc: 0.9426

Epoch 00172: val_acc did not improve from 0.95836
Epoch 173/250
7047/7047 [==============================] - 1s 172us/step - loss: 0.0194 - acc: 0.9942 - val_loss: 0.2428 - val_acc: 0.9311

Epoch 00173: val_acc did not improve from 0.95836
Epoch 174/250
7047/7047 [==============================] - 1s 17

7047/7047 [==============================] - 1s 175us/step - loss: 0.0192 - acc: 0.9943 - val_loss: 0.2636 - val_acc: 0.9210

Epoch 00212: val_acc did not improve from 0.95836
Epoch 213/250
7047/7047 [==============================] - 1s 172us/step - loss: 0.0179 - acc: 0.9953 - val_loss: 0.2200 - val_acc: 0.9411

Epoch 00213: val_acc did not improve from 0.95836
Epoch 214/250
7047/7047 [==============================] - 1s 172us/step - loss: 0.0197 - acc: 0.9945 - val_loss: 0.2132 - val_acc: 0.9440

Epoch 00214: val_acc did not improve from 0.95836
Epoch 215/250
7047/7047 [==============================] - 1s 170us/step - loss: 0.0178 - acc: 0.9945 - val_loss: 0.2173 - val_acc: 0.9375

Epoch 00215: val_acc did not improve from 0.95836
Epoch 216/250
7047/7047 [==============================] - 1s 174us/step - loss: 0.0208 - acc: 0.9929 - val_loss: 0.1944 - val_acc: 0.9440

Epoch 00216: val_acc did not improve from 0.95836
Epoch 217/250
7047/7047 [==============================] - 1s 17

In [9]:
temp_weights = [layer.get_weights() for layer in model.layers]

In [10]:
len(temp_weights)

23

In [20]:
MODEL_SAVE_FOLDER_PATH = './model/cnn_test'
if not os.path.exists(MODEL_SAVE_FOLDER_PATH):
  os.mkdir(MODEL_SAVE_FOLDER_PATH)

model_path = MODEL_SAVE_FOLDER_PATH + '.hdf5'

cb_checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_acc',
                                verbose=1, save_best_only=True)

In [21]:
X_train = dp_train.images[:]
X_test = dp_test.images[:]
Y_train = dp_train.label[:]
Y_test = dp_test.label[:]


Y_train = keras.utils.to_categorical(Y_train,num_classes=class_num, dtype='float32')
Y_test = keras.utils.to_categorical(Y_test,num_classes=class_num, dtype='float32')

In [22]:
input_1 = Input(shape=(28, 28, 1))

#x_1 = conv1_layer_2(input_1)
#x_1 = conv2_layer_2(x_1)

x_1 = Conv2D(32, kernel_size=3, padding="valid", activation = 'relu')(input_1)
x_1 = BatchNormalization()(x_1)
x_1 = ReLU()(x_1)
x_1 = Dropout(0.45)(x_1)

x_1 = Conv2D(64, kernel_size=3, padding="valid", activation = 'relu')(x_1)
x_1 = BatchNormalization()(x_1)
x_1 = ReLU()(x_1)
x_1 = Dropout(0.45)(x_1)

x_1 = Conv2D(128, kernel_size=3, padding="valid", activation = 'relu')(x_1)
x_1 = BatchNormalization()(x_1)
x_1 = ReLU()(x_1)
x_1 = Dropout(0.45)(x_1)

x_1 = Conv2D(256, kernel_size=3, padding="valid", activation = 'relu')(x_1)
x_1 = BatchNormalization()(x_1)
x_1 = ReLU()(x_1)
x_1 = Dropout(0.45)(x_1)

x_1 = GlobalAveragePooling2D()(x_1)
x_1 = Dropout(0.45)(x_1)

x_1 = Dense(class_num,activation='softmax')(x_1)

model_2 = Model(inputs=input_1, outputs=x_1)

In [23]:
model_2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
batch_normalization_18 (Batc (None, 26, 26, 32)        128       
_________________________________________________________________
re_lu_13 (ReLU)              (None, 26, 26, 32)        0         
_________________________________________________________________
dropout_21 (Dropout)         (None, 26, 26, 32)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 24, 24, 64)        18496     
_________________________________________________________________
batch_normalization_19 (Batc (None, 24, 24, 64)        256       
__________

In [24]:
model_2.compile(loss='categorical_crossentropy',
                optimizer=keras.optimizers.Adam(lr=0.001,epsilon=1e-04),
                metrics=['accuracy'])

results = model_2.fit(X_train,Y_train,epochs=250,batch_size=128,validation_data=(X_test,Y_test),callbacks=[cb_checkpoint,lr_reduce])

Train on 7047 samples, validate on 1393 samples
Epoch 1/250
7047/7047 [==============================] - 3s 478us/step - loss: 2.9047 - acc: 0.1646 - val_loss: 2.8796 - val_acc: 0.1271

Epoch 00001: val_acc improved from -inf to 0.12706, saving model to ./model/cnn_test.hdf5
Epoch 2/250
7047/7047 [==============================] - 2s 300us/step - loss: 2.1002 - acc: 0.4271 - val_loss: 2.2926 - val_acc: 0.2649

Epoch 00002: val_acc improved from 0.12706 to 0.26490, saving model to ./model/cnn_test.hdf5
Epoch 3/250
7047/7047 [==============================] - 2s 302us/step - loss: 1.5005 - acc: 0.6230 - val_loss: 1.6337 - val_acc: 0.5671

Epoch 00003: val_acc improved from 0.26490 to 0.56712, saving model to ./model/cnn_test.hdf5
Epoch 4/250
7047/7047 [==============================] - 2s 301us/step - loss: 1.0867 - acc: 0.7539 - val_loss: 1.4662 - val_acc: 0.6109

Epoch 00004: val_acc improved from 0.56712 to 0.61091, saving model to ./model/cnn_test.hdf5
Epoch 5/250
7047/7047 [========

7047/7047 [==============================] - 2s 305us/step - loss: 0.0604 - acc: 0.9861 - val_loss: 0.3760 - val_acc: 0.9124

Epoch 00041: val_acc did not improve from 0.91385
Epoch 42/250
7047/7047 [==============================] - 2s 298us/step - loss: 0.0520 - acc: 0.9881 - val_loss: 0.3830 - val_acc: 0.9146

Epoch 00042: val_acc improved from 0.91385 to 0.91457, saving model to ./model/cnn_test.hdf5
Epoch 43/250
7047/7047 [==============================] - 2s 295us/step - loss: 0.0579 - acc: 0.9854 - val_loss: 0.4278 - val_acc: 0.8930

Epoch 00043: val_acc did not improve from 0.91457
Epoch 44/250
7047/7047 [==============================] - 2s 305us/step - loss: 0.0563 - acc: 0.9860 - val_loss: 0.3929 - val_acc: 0.9095

Epoch 00044: val_acc did not improve from 0.91457
Epoch 45/250
7047/7047 [==============================] - 2s 304us/step - loss: 0.0517 - acc: 0.9884 - val_loss: 0.4260 - val_acc: 0.8894

Epoch 00045: val_acc did not improve from 0.91457
Epoch 46/250
7047/7047 [=

7047/7047 [==============================] - 2s 295us/step - loss: 0.0280 - acc: 0.9930 - val_loss: 0.4608 - val_acc: 0.9052

Epoch 00083: val_acc did not improve from 0.92965
Epoch 84/250
7047/7047 [==============================] - 2s 295us/step - loss: 0.0247 - acc: 0.9936 - val_loss: 0.4550 - val_acc: 0.9203

Epoch 00084: val_acc did not improve from 0.92965
Epoch 85/250
7047/7047 [==============================] - 2s 300us/step - loss: 0.0261 - acc: 0.9932 - val_loss: 0.4597 - val_acc: 0.9232

Epoch 00085: val_acc did not improve from 0.92965
Epoch 86/250
7047/7047 [==============================] - 2s 295us/step - loss: 0.0242 - acc: 0.9940 - val_loss: 0.4787 - val_acc: 0.9146

Epoch 00086: val_acc did not improve from 0.92965
Epoch 87/250
7047/7047 [==============================] - 2s 291us/step - loss: 0.0225 - acc: 0.9946 - val_loss: 0.4279 - val_acc: 0.9253

Epoch 00087: val_acc did not improve from 0.92965
Epoch 88/250
7047/7047 [==============================] - 2s 297us/s

7047/7047 [==============================] - 2s 286us/step - loss: 0.0170 - acc: 0.9959 - val_loss: 0.4564 - val_acc: 0.9225

Epoch 00126: val_acc did not improve from 0.93180
Epoch 127/250
7047/7047 [==============================] - 2s 282us/step - loss: 0.0166 - acc: 0.9965 - val_loss: 0.4564 - val_acc: 0.9268

Epoch 00127: val_acc did not improve from 0.93180
Epoch 128/250
7047/7047 [==============================] - 2s 281us/step - loss: 0.0175 - acc: 0.9960 - val_loss: 0.4522 - val_acc: 0.9225

Epoch 00128: val_acc did not improve from 0.93180
Epoch 129/250
7047/7047 [==============================] - 2s 284us/step - loss: 0.0166 - acc: 0.9959 - val_loss: 0.4726 - val_acc: 0.9203

Epoch 00129: val_acc did not improve from 0.93180
Epoch 130/250
7047/7047 [==============================] - 2s 284us/step - loss: 0.0157 - acc: 0.9966 - val_loss: 0.4519 - val_acc: 0.9218

Epoch 00130: val_acc did not improve from 0.93180
Epoch 131/250
7047/7047 [==============================] - 2s 28

7047/7047 [==============================] - 2s 282us/step - loss: 0.0132 - acc: 0.9970 - val_loss: 0.4648 - val_acc: 0.9282

Epoch 00169: val_acc did not improve from 0.93252
Epoch 170/250
7047/7047 [==============================] - 2s 280us/step - loss: 0.0148 - acc: 0.9969 - val_loss: 0.4612 - val_acc: 0.9253

Epoch 00170: val_acc did not improve from 0.93252
Epoch 171/250
7047/7047 [==============================] - 2s 281us/step - loss: 0.0147 - acc: 0.9967 - val_loss: 0.4594 - val_acc: 0.9253

Epoch 00171: val_acc did not improve from 0.93252
Epoch 172/250
7047/7047 [==============================] - 2s 281us/step - loss: 0.0166 - acc: 0.9952 - val_loss: 0.4654 - val_acc: 0.9253

Epoch 00172: val_acc did not improve from 0.93252
Epoch 173/250
7047/7047 [==============================] - 2s 281us/step - loss: 0.0137 - acc: 0.9963 - val_loss: 0.4608 - val_acc: 0.9268

Epoch 00173: val_acc did not improve from 0.93252
Epoch 174/250
7047/7047 [==============================] - 2s 28

7047/7047 [==============================] - 2s 292us/step - loss: 0.0129 - acc: 0.9969 - val_loss: 0.4620 - val_acc: 0.9275

Epoch 00212: val_acc did not improve from 0.93252
Epoch 213/250
7047/7047 [==============================] - 2s 298us/step - loss: 0.0125 - acc: 0.9972 - val_loss: 0.4613 - val_acc: 0.9268

Epoch 00213: val_acc did not improve from 0.93252
Epoch 214/250
7047/7047 [==============================] - 2s 286us/step - loss: 0.0128 - acc: 0.9973 - val_loss: 0.4619 - val_acc: 0.9268

Epoch 00214: val_acc did not improve from 0.93252
Epoch 215/250
7047/7047 [==============================] - 2s 296us/step - loss: 0.0136 - acc: 0.9965 - val_loss: 0.4610 - val_acc: 0.9268

Epoch 00215: val_acc did not improve from 0.93252
Epoch 216/250
7047/7047 [==============================] - 2s 286us/step - loss: 0.0125 - acc: 0.9969 - val_loss: 0.4602 - val_acc: 0.9268

Epoch 00216: val_acc did not improve from 0.93252
Epoch 217/250
7047/7047 [==============================] - 2s 28

In [25]:
temp_weights_2 = [layer.get_weights() for layer in model_2.layers]

In [26]:
input_1 = Input(shape=(28, 28, 1))

#x_1 = conv1_layer_2(input_1)
#x_1 = conv2_layer_2(x_1)

x_1 = Conv2D(32, kernel_size=3, padding="valid", activation = 'relu')(input_1)
x_1 = BatchNormalization()(x_1)
x_1 = ReLU()(x_1)
x_1 = Dropout(0.45)(x_1)

x_1 = Conv2D(64, kernel_size=3, padding="valid", activation = 'relu')(x_1)
x_1 = BatchNormalization()(x_1)
x_1 = ReLU()(x_1)
x_1 = Dropout(0.45)(x_1)

x_1 = Conv2D(128, kernel_size=3, padding="valid", activation = 'relu')(x_1)
x_1 = BatchNormalization()(x_1)
x_1 = ReLU()(x_1)
x_1 = Dropout(0.45)(x_1)

x_1 = Conv2D(256, kernel_size=3, padding="valid", activation = 'relu')(x_1)
x_1 = BatchNormalization()(x_1)
x_1 = ReLU()(x_1)
x_1 = Dropout(0.45)(x_1)

x_1 = GlobalAveragePooling2D()(x_1)
x_1 = Dropout(0.45)(x_1)


# lstm part
input_2 = Input(shape=(64, 2))

x_2 = Conv1D(32,kernel_size=4, padding='same',strides=1)(input_2)
x_2 = BatchNormalization(momentum=0.8)(x_2)
x_2 = LeakyReLU(0.2)(x_2)
x_2 = Dropout(0.5)(x_2)

x_2 = Conv1D(64,kernel_size=5, strides=1, padding='same')(x_2)
x_2 = BatchNormalization(momentum=0.8)(x_2)
x_2 = LeakyReLU(alpha=0.2)(x_2)
x_2 = Dropout(0.5)(x_2)

x_2 = Conv1D(128, kernel_size=6, strides=1, padding='same')(x_2)
x_2 = BatchNormalization(momentum=0.8)(x_2)
x_2 = LeakyReLU(0.2)(x_2)
x_2 = Dropout(0.5)(x_2)


x_2 = Conv1D(256, kernel_size=7, strides=1, padding='same')(x_2)
x_2 = BatchNormalization(momentum=0.8)(x_2)
x_2 = LeakyReLU(0.2)(x_2)
x_2 = Dropout(0.5)(x_2)

x_2 = Conv1D(512, kernel_size=8, strides=1, padding='same')(x_2)
x_2 = BatchNormalization(momentum=0.8)(x_2)
x_2 = LeakyReLU(0.2)(x_2)
x_2 = Dropout(0.5)(x_2)

x_2 = GlobalAveragePooling1D()(x_2)



merged = concatenate([x_1,x_2])

m = Dense(class_num, activation='softmax')(merged)

model_3 = Model(inputs=[input_1, input_2], outputs = m)


In [28]:
model_3.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 64, 2)        0                                            
__________________________________________________________________________________________________
conv1d_6 (Conv1D)               (None, 64, 32)       288         input_7[0][0]                    
__________________________________________________________________________________________________
batch_normalization_26 (BatchNo (None, 64, 32)       128         conv1d_6[0][0]                   
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
leaky_re_l

In [34]:
temp_weights_3 = [layer.get_weights() for layer in model_3.layers]
print(len(temp_weights_3))

43


In [45]:
for i in range(3):
    model_3.layers[i].set_weights(temp_weights[i])
for i, j in enumerate(range(4, 41, 2)):
    print(i,j)
    model_3.layers[j].set_weights(temp_weights[i+3])
for i, j in enumerate(range(3,41,2)):
    model_3.layers[j].set_weights(temp_weights_2[i])

0 4
1 6
2 8
3 10
4 12
5 14
6 16
7 18
8 20
9 22
10 24
11 26
12 28
13 30
14 32
15 34
16 36
17 38
18 40


In [46]:
# minmax 실행
train_x = dp_train.point[:,:,0].reshape((-1,1))
test_x = dp_test.point[:,:,0].reshape((-1,1))

train_y = dp_train.point[:,:,1].reshape((-1,1))
test_y = dp_test.point[:,:,1].reshape((-1,1))

data_x = scaler.fit_transform(train_x).reshape((-1,64,1))
data_test_x = scaler.transform(test_x).reshape((-1,64,1))

data_y = scaler_.fit_transform(train_y).reshape((-1,64,1))
data_test_y = scaler_.transform(test_y).reshape((-1,64,1))

data = np.hstack((data_x,data_y)).reshape((-1,64,2),order='F')
data_test = np.hstack((data_test_x,data_test_y)).reshape((-1,64,2),order='F')

#train set test set 나누기
X_train = data[:,:,:]
#X_train_ = copy.deepcopy(dp.point)
X_test = data_test[:,:,:]
X_test_ = copy.deepcopy(data[:,:,:])

X_train_img = dp_train.images[:]
X_test_img = dp_test.images[:]
Y_train = dp_train.label[:]
Y_test = dp_test.label[:]


Y_train = keras.utils.to_categorical(Y_train,num_classes=class_num, dtype='float32')
Y_test = keras.utils.to_categorical(Y_test,num_classes=class_num, dtype='float32')

/home/kist-student/anaconda3/envs/tensor/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/home/kist-student/anaconda3/envs/tensor/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [ ]:
model_3.compile(loss='categorical_crossentropy',
                optimizer=keras.optimizers.Adam(lr=0.001,epsilon=1e-04),
                metrics=['accuracy'])

hist_2 = model_3.fit([X_train_img,X_train],Y_train,epochs=250,batch_size=128,validation_data=([X_test_img,X_test],Y_test),callbacks=[lr_reduce])

Train on 7047 samples, validate on 1393 samples
Epoch 1/250
7047/7047 [==============================] - 6s 856us/step - loss: 0.8708 - acc: 0.8104 - val_loss: 0.2622 - val_acc: 0.9440
Epoch 2/250
7047/7047 [==============================] - 3s 454us/step - loss: 0.1210 - acc: 0.9844 - val_loss: 0.2470 - val_acc: 0.9311
Epoch 3/250
7047/7047 [==============================] - 3s 453us/step - loss: 0.0696 - acc: 0.9913 - val_loss: 0.2076 - val_acc: 0.9368
Epoch 4/250
7047/7047 [==============================] - 3s 454us/step - loss: 0.0504 - acc: 0.9936 - val_loss: 0.1967 - val_acc: 0.9375
Epoch 5/250
7047/7047 [==============================] - 3s 453us/step - loss: 0.0384 - acc: 0.9957 - val_loss: 0.2296 - val_acc: 0.9304
Epoch 6/250
7047/7047 [==============================] - 3s 452us/step - loss: 0.0312 - acc: 0.9962 - val_loss: 0.2225 - val_acc: 0.9311
Epoch 7/250
7047/7047 [==============================] - 3s 453us/step - loss: 0.0393 - acc: 0.9936 - val_loss: 0.2078 - val_acc: 

In [ ]:
score = model_3.evaluate([X_test_img,X_test], Y_test, verbose=0)
pred = model_3.predict([X_test_img,X_test])
true_value = np.argmax(Y_test,1)
predict_value = np.argmax(pred,1)
print(np.argmax(pred,axis=1))
print(score[1]*100)
#print("복원된 모델의 정확도: {:5.2f}%".format(100*score))
list_ = []
for i in range(np.size(true_value,0)):
  if(true_value[i] != predict_value[i]):
    if(number):
      print('t:{},p:{}'.format(true_value[i],predict_value[i]))
    else:
      print('t:{},p:{}'.format(chr(97+true_value[i]),chr(97+predict_value[i])))
    list_.append(i)